**Colab Link URL**
- https://colab.research.google.com/drive/1HIGLcY2PICk2BE_aym9noJcPxhY4Q8Vb?usp=sharing

이 노트북은 [Google Colab](https://colab.research.google.com/)에서 실행하시기에 최적화되어있습니다.
상기 URL을 이용해서 코드를 실행 해보세요.

In [12]:
!pip install -U -q langchain langchain-community langchain-openai networkx

In [16]:
import os

from langchain_openai import ChatOpenAI
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph
from langchain_community.graphs.networkx_graph import KnowledgeTriple
from langchain.chains import GraphQAChain
from google.colab import userdata

In [17]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [18]:
graph = NetworkxEntityGraph()

# 그래프에 샘플 데이터 주입
# (주어, 관계, 목적어) 형태의 트리플을 KnowledgeTriple 객체로 추가한다.
graph.add_triple(KnowledgeTriple("톰 행크스", "ACTED_IN", "포레스트 검프"))
graph.add_triple(KnowledgeTriple("톰 행크스", "ACTED_IN", "캐스트 어웨이"))
graph.add_triple(KnowledgeTriple("톰 행크스", "ACTED_IN", "라이언 일병 구하기"))
graph.add_triple(KnowledgeTriple("로버트 저메키스", "DIRECTED", "포레스트 검프"))
graph.add_triple(KnowledgeTriple("로버트 저메키스", "DIRECTED", "캐스트 어웨이"))
graph.add_triple(KnowledgeTriple("스티븐 스필버그", "DIRECTED", "라이언 일병 구하기"))
print("--- NetworkX 지식 그래프에 샘플 데이터 주입 완료 ---")

--- NetworkX 지식 그래프에 샘플 데이터 주입 완료 ---


In [19]:
# 그래프 트리플 확인
print("그래프 트리플:")
print(graph.get_triples())

그래프 트리플:
[('톰 행크스', '포레스트 검프', 'ACTED_IN'), ('톰 행크스', '캐스트 어웨이', 'ACTED_IN'), ('톰 행크스', '라이언 일병 구하기', 'ACTED_IN'), ('로버트 저메키스', '포레스트 검프', 'DIRECTED'), ('로버트 저메키스', '캐스트 어웨이', 'DIRECTED'), ('스티븐 스필버그', '라이언 일병 구하기', 'DIRECTED')]


In [20]:
# LLM과 그래프를 연결하여 자연어 질문을 그래프 탐색으로 변환하고 실행하는 체인 생성
llm = ChatOpenAI(
    model="gpt-5-mini",
    openai_api_key=OPENAI_API_KEY,
)
chain = GraphQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
)
print("--- Text-to-Graph QA 체인 생성 완료 ---")

--- Text-to-Graph QA 체인 생성 완료 ---


In [21]:
# 체인 실행 및 결과 분석
print("\n--- [사례 1] 단일 관계 질문 실행 ---")
question1 = "톰 행크스가 출연한 영화는 무엇인가요?"
response1 = chain.invoke({"query": question1})
print("\n최종 답변 1:")
print(response1["result"])

print("\n" + "="*50 + "\n")


--- [사례 1] 단일 관계 질문 실행 ---


> Entering new GraphQAChain chain...
Entities Extracted:
톰 행크스
Full Context:
톰 행크스 ACTED_IN 포레스트 검프
톰 행크스 ACTED_IN 캐스트 어웨이
톰 행크스 ACTED_IN 라이언 일병 구하기

> Finished chain.

최종 답변 1:
톰 행크스가 출연한 영화는 다음과 같습니다: 포레스트 검프, 캐스트 어웨이, 라이언 일병 구하기.




In [22]:
print("--- [사례 2] 다중 홉(Multi-hop) 관계 질문 실행 ---")
question2 = "로버트 저메키스가 감독하고 톰 행크스가 출연한 영화는 무엇인가요?"
response2 = chain.invoke({"query": question2})
print("\n최종 답변 2:")
print(response2["result"])

--- [사례 2] 다중 홉(Multi-hop) 관계 질문 실행 ---


> Entering new GraphQAChain chain...
Entities Extracted:
로버트 저메키스, 톰 행크스
Full Context:
로버트 저메키스 DIRECTED 포레스트 검프
로버트 저메키스 DIRECTED 캐스트 어웨이
톰 행크스 ACTED_IN 포레스트 검프
톰 행크스 ACTED_IN 캐스트 어웨이
톰 행크스 ACTED_IN 라이언 일병 구하기

> Finished chain.

최종 답변 2:
로버트 저메키스가 감독하고 톰 행크스가 출연한 영화는 "포레스트 검프"와 "캐스트 어웨이"입니다.
